In [94]:
import requests
from bs4 import BeautifulSoup
import os
from multiprocessing.dummy import Pool
import json
from tqdm.notebook import tqdm
import time

`check_links` - проверяет полноту парсинга ссылок на книги данного автора

In [78]:
def check_links(result, links):
    links_total = int(result.find('span', class_='rd-listing-count__total').get_text())
    return links_total == len(links)

In [110]:
!ls

data                  parsing               spotify-api-token.key
images                seminar.ipynb
main.ipynb            sphere-cookies.json


`get_urls` - возвращает `tuple` из `id` автора и списка ссылок на его книги. Если список неполный, к `id` дописывает `_not_full`.

In [79]:
def get_links(author):
    
    url = 'https://www.respublica.ru/authors/'
    page = 0
    links = []
    
    while True:
        page += 1
        link = url + author + '?page=' + str(page)
            try:
                response = requests.get(link, timeout=30)
            except:
                with open('')
        result = BeautifulSoup(response.content, 'html.parser')
        last_page = result.find_all('a',
                                    class_='rd-listing-pagination__link')[-1].get_text()
        if int(last_page) < page:
            if  not check_links(result, links):
                author += '_not_full'
            return (author, links)
        link_blocks = result.find_all('a',
                                      class_='rd-listing-product-item__image-wrapper')
        links += [link['href'] for link in link_blocks]

In [84]:
def progresser(n):
    interval = 0.001 / (n + 2)
    total = 5000
    text = "#{}, est. {:<04.2}s".format(n, interval * total)
    for _ in trange(total, desc=text, position=n):
        sleep(interval)

Читаем список `id` авторов из файла и запускаем мультипроцессорный парсинг функцией `get_links`, преобразуем результат в словарь с ключами `id` авторов и значениями списками ссылок на книги.

In [95]:
with open('authors.txt', 'r') as f:
    authors = f.read().split('\n')
    
with Pool(processes=4) as pool:
    links = dict(tqdm(pool.imap(get_links, authors), total=len(authors)))

Проверяем полноту парсинга для всех авторов.

In [97]:
links_full = True
for item_link in links:
    if item_link.endswith('_not_full'):
        links_full = False
        print(item_link)
if links_full:
    print('All links are parsed')
else:
    print('Some links are not parsed')

All links are parsed


Сохраняем ссылки в файл.

In [73]:
with open('links.json', 'w') as f:
    json.dump(dict(links), f)